In [58]:
import re
import pickle
import pandas as pd
import numpy as np
import csv


In [59]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [60]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,average_precision_score,recall_score


In [61]:
#PREPROCESSING
def preprocess(col):
    col_l = []
    for r in col:
        #print(r)
        try:
            a = r.lower() 
            a = a.replace("'s", "")
            #a = a.replace(["\n"], "hjfgjh")
            a = re.sub('[^a-zA-Z0-9 \n\.]', '',a)
            a = re.sub('[:.\n\t]', '',a)
            a = re.sub('[?:!.+,]]][]]//;]', '',a) 
            
            col_l.append(a)
        except:
            #print("No header found")
            col_l.append("No Header")
            pass

    # Downloading punkt and wordnet from NLTK
    nltk.download('punkt')
    print("------------------------------------------------------------")
    nltk.download('wordnet')
    
    wordnet_lemmatizer = WordNetLemmatizer()
    
    nltk.download('stopwords')
    
    lemmatized_text_list = []

    for s in col_l:
        #print(s)
        # Create an empty list containing lemmatized words
        lemmatized_list = []

        # Save the text and its words into an object
        #text = df.loc[row]['Content_Parsed_4']
        text_words = s.split(" ")

        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

        # Join the list
        lemmatized_text = " ".join(lemmatized_list)

        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)
    
    new_lemmatized_text_list = []
    for x in lemmatized_text_list:
        new_lemmatized_text_list.append(" ".join(x.split())) 
    
    pl = pd.Series(new_lemmatized_text_list)
   # pl = pd.Series(col_l)
    stop_words = list(stopwords.words('english'))
    
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        #pl = pd.Series(col_l)
   
        pl = pl.str.replace(regex_stopword, '')
        
    
    return pl.tolist()

In [62]:
############Read the new dataset for prediction#########

In [63]:

#read the csv file which containd the data for prediction
#dfnew= pd.read_csv('Articles.csv')#100  non iot articles
#dfnew= pd.read_csv('Products.csv')#100  non iot articles
dfnew= pd.read_csv('datasetiotnoniot233label.csv')#233 iot and  noniot projects for prediction 


In [64]:
#convert data frame into a list 
cnew=(dfnew['descr']).tolist()
lnew=(dfnew['label']).tolist()# used only for finding the accuracy

In [65]:
connew=preprocess(cnew)
print(connew)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


------------------------------------------------------------
['   lot  work  image process  need  test cameras imagers   go  create  scalable test platform read    project     lot  work  image process  need  test cameras imagers   go  create  scalable test platform   look  create image process solutions previously  use image sensors  use  parallel interface  cameras  use hdmi interfaces one common interface  image sensors  mipi  let take  look     build  image process design   zybo z7   capable  use  hdmi input   mipi camera interface  also want  lay  grind work   able  create  image process system  allow   quickly  easily test cameras  sensors    future upgrade   project    support  parallel pmod camera  well     able  select   imager  wish  use   algorithm    add advantage   pin    zybo mipi connector   align   commonly use maker cameras  example  pi cameras   look     use  pi camera  another project mipi   commonly use abbreviation   mobile industry processor interface    interface 

In [66]:
df1 = pd.DataFrame({"descr":connew})
df1

descr
0       lot  work  image process  need  test camera...
1     project  want  share   today   arduino maze g...
2     pantry   automatically replenish kitchen item...
3      make lanethernet relay switch use arduino un...
4    cellular backdoor  hologram nova  raspberry pi...
..                                                 ...
228  ingredients 500g chicken breast 250ml cream 20...
229  homemade bacon   truly magical piece  meat mak...
230  simple fast  delicious im try  lowcarb diet  w...
231  materials teachers   use  instructable   class...
232  ingredients 500g chicken breast 300ml cream 15...

[233 rows x 1 columns]

In [67]:
label=pd.DataFrame({"label":lnew})
label

label
0        1
1        1
2        1
3        1
4        1
..     ...
228      0
229      0
230      0
231      0
232      0

[233 rows x 1 columns]

In [68]:
#loading the tfidf vectorizer
tf2 = pickle.load(open("tfidf1.pkl", 'rb'))

In [69]:
#apply tfidf for testing data and just use transform method instead of fit_transform
pred_test= tf2.transform(connew).toarray()
print(pred_test)
        

[[0.         0.07388589 0.1406397  ... 0.11856445 0.         0.        ]
 [0.         0.         0.09934618 ... 0.         0.         0.        ]
 [0.05271676 0.05900858 0.05352164 ... 0.09469087 0.         0.        ]
 ...
 [0.12231373 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [70]:
################################## DECISION TREE ###############################################

In [71]:
# load the model from disk
loaded_model = pickle.load(open("finalized_model.sav", 'rb'))
DTC_pred1 = loaded_model.predict(pred_test)
#accuracy
print("Decision Tree prediction Accuracy Score  ->",accuracy_score(label, loaded_model.predict(pred_test))*100)
#confusion matrix
matrix = confusion_matrix(label, loaded_model.predict(pred_test))
#displaying probability values
y=loaded_model.predict_proba(pred_test)
print(y)
#to show the order of the output.here 0 first then 1
p=loaded_model.classes_
print(p)
print("confusion matrix")
print(matrix)
DTC_pred1#prining the output array

Decision Tree prediction Accuracy Score  -> 99.57081545064378
[[0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [72]:
loaded_naive = pickle.load(open("finalized_naive.sav", 'rb'))

Naive_pred1 = loaded_naive.predict(pred_test)
print("Naive Bayes prediction Accuracy Score  ->",accuracy_score(label, loaded_naive.predict(pred_test))*100)
y=loaded_naive.predict_proba(pred_test)
print(y)
p=loaded_naive.classes_
print(p)
matrix = confusion_matrix(label, loaded_naive.predict(pred_test))
print("confusion matrix")
print(matrix)
Naive_pred1

Naive Bayes prediction Accuracy Score  -> 98.71244635193133
[[2.64807251e-03 9.97351927e-01]
 [3.41490177e-02 9.65850982e-01]
 [1.14982157e-03 9.98850178e-01]
 [1.45764997e-03 9.98542350e-01]
 [1.81523341e-04 9.99818477e-01]
 [8.36996682e-05 9.99916300e-01]
 [1.89147623e-03 9.98108524e-01]
 [3.80087863e-03 9.96199121e-01]
 [1.01480290e-03 9.98985197e-01]
 [1.01412544e-01 8.98587456e-01]
 [4.39047325e-01 5.60952675e-01]
 [8.87299242e-02 9.11270076e-01]
 [4.81712956e-03 9.95182870e-01]
 [4.19186759e-03 9.95808132e-01]
 [5.30561461e-01 4.69438539e-01]
 [3.31843456e-04 9.99668157e-01]
 [4.01704774e-01 5.98295226e-01]
 [7.47050438e-02 9.25294956e-01]
 [2.64583208e-02 9.73541679e-01]
 [8.00457098e-03 9.91995429e-01]
 [5.28903078e-03 9.94710969e-01]
 [2.64732287e-03 9.97352677e-01]
 [7.88961441e-05 9.99921104e-01]
 [7.05621967e-03 9.92943780e-01]
 [5.21770921e-04 9.99478229e-01]
 [2.13984864e-02 9.78601514e-01]
 [1.67448235e-03 9.98325518e-01]
 [5.37387797e-03 9.94626122e-01]
 [6.76238834e-04

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [73]:
loaded_svm = pickle.load(open("finalized_svm.sav", 'rb'))
print("SVM prediction Accuracy Score  ->",accuracy_score(label, loaded_svm.predict(pred_test))*100)
svm_pred1 = loaded_svm.predict(pred_test)
y=loaded_svm.predict_proba(pred_test)
print(y)
p=loaded_svm.classes_
print(p)
matrix = confusion_matrix(label, loaded_svm.predict(pred_test))
print("confusion matrix")
print(matrix)
svm_pred1


SVM prediction Accuracy Score  -> 99.14163090128756
[[2.07832711e-02 9.79216729e-01]
 [3.18522525e-06 9.99996815e-01]
 [2.23709512e-02 9.77629049e-01]
 [9.03262248e-03 9.90967378e-01]
 [5.14313897e-08 9.99999949e-01]
 [9.17018576e-10 9.99999999e-01]
 [5.77262950e-10 9.99999999e-01]
 [1.85470584e-08 9.99999981e-01]
 [3.56688711e-06 9.99996433e-01]
 [1.29730506e-02 9.87026949e-01]
 [1.14838208e-01 8.85161792e-01]
 [7.71496354e-02 9.22850365e-01]
 [4.68470271e-03 9.95315297e-01]
 [6.65607346e-08 9.99999933e-01]
 [3.04037725e-01 6.95962275e-01]
 [7.35775176e-09 9.99999993e-01]
 [1.02248270e-01 8.97751730e-01]
 [2.36818531e-02 9.76318147e-01]
 [5.27151340e-03 9.94728487e-01]
 [3.88935666e-03 9.96110643e-01]
 [1.55252271e-05 9.99984475e-01]
 [2.17767337e-02 9.78223266e-01]
 [5.92046670e-07 9.99999408e-01]
 [1.13150194e-05 9.99988685e-01]
 [7.99703275e-10 9.99999999e-01]
 [1.52561789e-02 9.84743821e-01]
 [2.12041650e-07 9.99999788e-01]
 [1.29143228e-06 9.99998709e-01]
 [1.20619795e-06 9.99998

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [74]:
loaded_knn = pickle.load(open("finalized_knn.sav", 'rb'))
knn_pred1 = loaded_knn.predict(pred_test)
print("KNN prediction Accuracy Score  ->",accuracy_score(label, loaded_knn.predict(pred_test))*100)
y=loaded_knn.predict_proba(pred_test)
print(y)
p=loaded_knn.classes_
print(p)
matrix = confusion_matrix(label, loaded_knn.predict(pred_test))
print("confusion matrix")
print(matrix)
knn_pred1


KNN prediction Accuracy Score  -> 96.99570815450643
[[0.14253896 0.85746104]
 [0.14559596 0.85440404]
 [0.         1.        ]
 [0.14561859 0.85438141]
 [0.         1.        ]
 [0.         1.        ]
 [0.15000374 0.84999626]
 [0.14421721 0.85578279]
 [0.         1.        ]
 [0.16130005 0.83869995]
 [0.71687838 0.28312162]
 [0.71642388 0.28357612]
 [0.         1.        ]
 [0.14640804 0.85359196]
 [0.85831159 0.14168841]
 [0.         1.        ]
 [0.71397729 0.28602271]
 [0.29647905 0.70352095]
 [0.15565615 0.84434385]
 [0.14397201 0.85602799]
 [0.28596707 0.71403293]
 [0.15140056 0.84859944]
 [0.         1.        ]
 [0.15273454 0.84726546]
 [0.14344344 0.85655656]
 [0.         1.        ]
 [0.14964434 0.85035566]
 [0.29241463 0.70758537]
 [0.         1.        ]
 [0.14468897 0.85531103]
 [0.         1.        ]
 [0.14083642 0.85916358]
 [0.         1.        ]
 [0.         1.        ]
 [0.14231061 0.85768939]
 [0.13619391 0.86380609]
 [0.14389402 0.85610598]
 [0.14974442 0.85025558

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [75]:
loaded_sgd = pickle.load(open("finalized_sgd.sav", 'rb'))
sgd_pred1 = loaded_sgd.predict(pred_test)
print("SGD prediction Accuracy Score  ->",accuracy_score(label, loaded_sgd.predict(pred_test))*100)
y=loaded_sgd.predict_proba(pred_test)
print(y)
p=loaded_sgd.classes_
print(p)
matrix = confusion_matrix(label, loaded_sgd.predict(pred_test))
print("confusion matrix")
print(matrix)
sgd_pred1


SGD prediction Accuracy Score  -> 99.14163090128756
[[1.18947239e-01 8.81052761e-01]
 [2.19691928e-02 9.78030807e-01]
 [1.08448338e-01 8.91551662e-01]
 [7.73731355e-02 9.22626865e-01]
 [1.17155678e-02 9.88284432e-01]
 [2.60026058e-03 9.97399739e-01]
 [2.18263474e-03 9.97817365e-01]
 [4.53067443e-03 9.95469326e-01]
 [1.64802170e-02 9.83519783e-01]
 [4.61369760e-02 9.53863024e-01]
 [1.80366628e-01 8.19633372e-01]
 [1.28850997e-01 8.71149003e-01]
 [3.92599498e-02 9.60740050e-01]
 [7.68557653e-03 9.92314423e-01]
 [4.12740506e-01 5.87259494e-01]
 [3.91493731e-03 9.96085063e-01]
 [2.02329951e-01 7.97670049e-01]
 [1.19596459e-01 8.80403541e-01]
 [3.68195717e-02 9.63180428e-01]
 [5.78276611e-02 9.42172339e-01]
 [3.68868983e-02 9.63113102e-01]
 [6.50815975e-02 9.34918403e-01]
 [1.53277242e-02 9.84672276e-01]
 [2.88803100e-02 9.71119690e-01]
 [2.39170538e-03 9.97608295e-01]
 [7.87329392e-02 9.21267061e-01]
 [9.91114955e-03 9.90088850e-01]
 [1.20287068e-02 9.87971293e-01]
 [1.34951874e-02 9.86504

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [76]:
loaded_random = pickle.load(open("finalized_random.sav", 'rb'))
random_pred1 = loaded_random.predict(pred_test)
random_pred1
print("Random Forest prediction Accuracy Score  ->",accuracy_score(label, loaded_random.predict(pred_test))*100)
y=loaded_random.predict_proba(pred_test)
print(y)
p=loaded_random.classes_
print(p)
matrix = confusion_matrix(label, loaded_random.predict(pred_test))
print("confusion matrix")
print(matrix)
random_pred1


Random Forest prediction Accuracy Score  -> 100.0
[[0.29661017 0.70338983]
 [0.         1.        ]
 [0.         1.        ]
 [0.49418605 0.50581395]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.43333333 0.56666667]
 [0.         1.        ]
 [0.0483871  0.9516129 ]
 [0.         1.        ]
 [0.36       0.64      ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.29661017 0.70338983]
 [0.         1.        ]
 [0.         1.        ]
 [0.11904762 0.88095238]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.0483871  0.9516129 ]
 [0.015625   0.984375  ]
 [0.36       0.64      ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.43333333 0.56666667]


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)